In [ ]:
try:
    from openmdao.utils.notebook_utils import notebook_mode
except ImportError:
    !python -m pip install openmdao[notebooks]

In [ ]:
from openmdao.utils.assert_utils import assert_near_equal

import os
if os.path.exists('cases.sql'):
    os.remove('cases.sql')

# Accessing Recorded Metadata

In addition to the cases themselves, a `CaseReader` may also record
certain metadata about the model and its constituent systems and solvers.

## Problem Metadata

By default, a case recorder will save metadata about the model to assist in later visualization
and debugging.  This information is made available via the `problem_metadata` attribute of
a `CaseReader`.

In [ ]:
from openmdao.utils.notebook_utils import get_code
from myst_nb import glue
glue("code_src84", get_code("openmdao.test_suite.components.sellar.SellarDerivatives"), display=False)

:::{Admonition} `SellarDerivatives` class definition 
:class: dropdown

{glue:}`code_src84`
:::

In [ ]:
import openmdao.api as om
from openmdao.test_suite.components.sellar import SellarDerivatives

prob = om.Problem(SellarDerivatives())

recorder = om.SqliteRecorder("cases.sql")
prob.driver.add_recorder(recorder)

prob.setup()
prob.run_driver()
prob.cleanup()

cr = om.CaseReader("cases.sql")

In [ ]:
# access list of connections stored in metadata
cr.problem_metadata['connections_list']

In [ ]:
expected = [
    {'src': '_auto_ivc.v0', 'tgt': 'd1.z'},
    {'src': '_auto_ivc.v0', 'tgt': 'd2.z'},
    {'src': '_auto_ivc.v0', 'tgt': 'obj_cmp.z'},
    {'src': '_auto_ivc.v1', 'tgt': 'd1.x'},
    {'src': '_auto_ivc.v1', 'tgt': 'obj_cmp.x'},
    {'src': 'd1.y1', 'tgt': 'con_cmp1.y1'},
    {'src': 'd1.y1', 'tgt': 'd2.y1', 'cycle_arrows': [[0, 1]]},
    {'src': 'd1.y1', 'tgt': 'obj_cmp.y1'},
    {'src': 'd2.y2', 'tgt': 'con_cmp2.y2'},
    {'src': 'd2.y2', 'tgt': 'd1.y2', 'cycle_arrows': [[1, 0]]},
    {'src': 'd2.y2', 'tgt': 'obj_cmp.y2'}
]

connections = sorted(cr.problem_metadata['connections_list'], key=lambda x: (x['src'], x['tgt']))
for i, meta in enumerate(connections):
    for key in meta:
        if key != 'cycle_arrows':
            assert meta[key] == expected[i][key]

In [ ]:
# access the model tree stored in metadata
cr.problem_metadata['tree']

In [ ]:
assert set(cr.problem_metadata['tree'].keys()) == {
    'name', 'type', 'class', 'expressions', 'component_type',
    'subsystem_type', 'is_parallel', 'linear_solver', 'linear_solver_options',
    'nonlinear_solver', 'nonlinear_solver_options', 'children', 'options'
}

assert cr.problem_metadata['tree']['name'] == 'root'

assert set([child["name"] for child in cr.problem_metadata['tree']["children"]]) == {
    '_auto_ivc', 'con_cmp1', 'con_cmp2', 'd1', 'd2', 'obj_cmp'
}

## System Options

All case recorders record the component options and scaling factors for all systems in the model.

These values are accessible using the `list_model_options` function of a case reader object.
This function displays and returns a dictionary of the option values for each system in the model.

If the model has been run multiple times, you can specify the run for which to get/display options.

In [ ]:
from openmdao.utils.notebook_utils import get_code
from myst_nb import glue
glue("code_src85", get_code("openmdao.test_suite.components.sellar.SellarDerivativesGrouped"), display=False)

:::{Admonition} `SellarDerivativesGrouped` class definition 
:class: dropdown

{glue:}`code_src85`
:::

In [ ]:
import openmdao.api as om
from openmdao.test_suite.components.sellar import SellarDerivativesGrouped

prob = om.Problem(model=SellarDerivativesGrouped())
prob.add_recorder(om.SqliteRecorder("cases.sql"))

# set option and run model
prob.model.options['nl_maxiter'] = 1
prob.setup()
prob.run_model()

# change option and run again
prob.model.options['nl_maxiter'] = 9
prob.setup()
prob.run_model()

# clean up after runs and open a case reader
prob.cleanup()
cr = om.CaseReader("cases.sql")

In [ ]:
# get/display options for initial run
options = cr.list_model_options()

In [ ]:
assert sorted(options.keys()) == sorted([
    'root', '_auto_ivc', 'con_cmp1', 'con_cmp2', 'mda', 'mda.d1', 'mda.d2', 'obj_cmp'
])

assert sorted(options['mda.d1'].keys()) == sorted(prob.model.mda.d1.options._dict.keys())

assert options['root']['nl_maxiter'] == 1

In [ ]:
# check nl_maxiter option for the second run
options = cr.list_model_options(run_number=1, out_stream=None)
options['root']['nl_maxiter']

In [ ]:
assert options['root']['nl_maxiter'] == 9

## Solver Options

All case recorders record the solver options for all solvers in the model.

These values are accessible using the `list_solver_options` function of a case reader object.

This function displays and returns a dictionary of the option values for each solver in the model.
If the model has been run multiple times, you can specify the run for which to get/display options.

In [ ]:
import openmdao.api as om
from openmdao.test_suite.components.sellar import SellarDerivativesGrouped

# configure a Newton solver with linesearch for the Sellar MDA Group
newton = om.NewtonSolver(solve_subsystems=True, max_sub_solves=4)
newton.linesearch = om.BoundsEnforceLS()

model = SellarDerivativesGrouped(mda_nonlinear_solver=newton)

prob = om.Problem(model)
prob.add_recorder(om.SqliteRecorder("cases.sql"))
prob.setup()

# initial run
newton.linesearch.options['bound_enforcement'] = 'vector'
prob.run_model()

# change linesearch and run again
newton.linesearch.options['bound_enforcement'] = 'wall'
prob.run_model()

# clean up after runs and open a case reader
prob.cleanup()
cr = om.CaseReader("cases.sql")

In [ ]:
# get/display options for initial run
options = cr.list_solver_options()

In [ ]:
print(sorted(options.keys()))

In [ ]:
print(options['root.NonlinearBlockGS']['maxiter'])

In [ ]:
print(options['root.ScipyKrylov']['maxiter'])

In [ ]:
print(options['mda.NewtonSolver']['maxiter'])

In [ ]:
print(options['mda.NewtonSolver']['solve_subsystems'])

In [ ]:
print(options['mda.NewtonSolver']['max_sub_solves'])

In [ ]:
print(options['mda.BoundsEnforceLS']['bound_enforcement'])

In [ ]:
assert sorted(options.keys()) == [
    'mda.BoundsEnforceLS', 'mda.NewtonSolver', 'mda.ScipyKrylov',
    'root.NonlinearBlockGS', 'root.ScipyKrylov'
]
assert options['root.NonlinearBlockGS']['maxiter'] == 10
assert options['root.ScipyKrylov']['maxiter'] == 1000
assert options['mda.NewtonSolver']['maxiter'] == 10
assert options['mda.NewtonSolver']['solve_subsystems'] == True
assert options['mda.NewtonSolver']['max_sub_solves'] == 4
assert options['mda.BoundsEnforceLS']['bound_enforcement'] == 'vector'

In [ ]:
# get options for second run
options = cr.list_solver_options(run_number=1, out_stream=None)
print(options['mda.BoundsEnforceLS']['bound_enforcement'])

In [ ]:
assert options['mda.BoundsEnforceLS']['bound_enforcement'] == 'wall'